In [1]:
import requests
from bs4 import BeautifulSoup
from numpy import random
from time import sleep
import pandas as pd

In [2]:
season_links = []
main_page = requests.get('https://j-archive.com/listseasons.php')
parsed_main = BeautifulSoup(main_page.content, 'html.parser')
main_body = parsed_main.find('div', id = 'content')
for a in main_body.find_all('a', href=True): 
    if a.text: 
        season_links.append('https://j-archive.com/'+a['href'])
season_links

['https://j-archive.com/showseason.php?season=38',
 'https://j-archive.com/showseason.php?season=37',
 'https://j-archive.com/showseason.php?season=goattournament',
 'https://j-archive.com/showseason.php?season=36',
 'https://j-archive.com/showseason.php?season=35',
 'https://j-archive.com/showseason.php?season=34',
 'https://j-archive.com/showseason.php?season=33',
 'https://j-archive.com/showseason.php?season=32',
 'https://j-archive.com/showseason.php?season=31',
 'https://j-archive.com/showseason.php?season=30',
 'https://j-archive.com/showseason.php?season=29',
 'https://j-archive.com/showseason.php?season=28',
 'https://j-archive.com/showseason.php?season=27',
 'https://j-archive.com/showseason.php?season=26',
 'https://j-archive.com/showseason.php?season=25',
 'https://j-archive.com/showseason.php?season=24',
 'https://j-archive.com/showseason.php?season=23',
 'https://j-archive.com/showseason.php?season=22',
 'https://j-archive.com/showseason.php?season=21',
 'https://j-archive

In [5]:
show_links = []
season_page = requests.get('https://j-archive.com/showseason.php?season=1')
parsed_season = BeautifulSoup(season_page.content, 'html.parser')
season_body = parsed_season.find('div', id='content')
for a in season_body.find_all('a', href=True): 
    if a.text and a['href'].startswith('http'): 
        show_links.append(a['href'])
show_links

['https://www.j-archive.com/showgame.php?game_id=7072',
 'https://www.j-archive.com/showgame.php?game_id=7071',
 'https://www.j-archive.com/showgame.php?game_id=7067',
 'https://www.j-archive.com/showgame.php?game_id=7061',
 'https://www.j-archive.com/showgame.php?game_id=5525',
 'https://www.j-archive.com/showgame.php?game_id=6936',
 'https://www.j-archive.com/showgame.php?game_id=4529',
 'https://www.j-archive.com/showgame.php?game_id=5120',
 'https://www.j-archive.com/showgame.php?game_id=5587',
 'https://www.j-archive.com/showgame.php?game_id=6559',
 'https://www.j-archive.com/showgame.php?game_id=4592',
 'https://www.j-archive.com/showgame.php?game_id=368',
 'https://www.j-archive.com/showgame.php?game_id=6490',
 'https://www.j-archive.com/showgame.php?game_id=6487',
 'https://www.j-archive.com/showgame.php?game_id=5018',
 'https://www.j-archive.com/showgame.php?game_id=5544',
 'https://www.j-archive.com/showgame.php?game_id=5009',
 'https://www.j-archive.com/showgame.php?game_id=

In [105]:
def get_categories(jeopardy_round):
    categories = []
    for c in jeopardy_round.find_all('td', class_='category_name'):
        categories.append(c.get_text().title())
    
    return categories
    
def clue_parser(jeopardy_round):
    rows_to_add = []
    
    for c in jeopardy_round.find_all('td', class_='clue_text'):
        rows_to_add.append([categories[int(c.get('id')[-3:-2])-1], c.get_text()])

    for i, c in enumerate(jeopardy_round.find_all('div', onmouseover=True)):
        c = BeautifulSoup(c.get('onmouseover'))
        rows_to_add[i].append(c.find('em', class_='correct_response').get_text())
        if c.find('td', class_='right'):
            rows_to_add[i].append(name_lookup[c.find('td', class_='right').get_text()])
        else:
            rows_to_add[i].append(None)
        if c.find_all('td', class_='wrong') and c.find_all('td', class_='wrong')[-1].get_text() == 'Triple Stumper':
            rows_to_add[i].append(True)
        else:
            rows_to_add[i].append(False)
        if c.find_all('td', class_='wrong') and len(c.find_all('td', class_='wrong')) == 4:
            rows_to_add[i].append(True)
        else:
            rows_to_add[i].append(False)
     
    for i, c in enumerate(jeopardy_round.find_all('table', class_='clue_header')):
        if c.find('td', class_='clue_value'):
            rows_to_add[i].append(c.find('td', class_='clue_value').get_text())
            rows_to_add[i].append(False)
        else:
            rows_to_add[i].append(c.find('td', class_='clue_value_daily_double').get_text())
            rows_to_add[i].append(True)
            
    for i, c in enumerate(jeopardy_round.find_all('td', class_='clue_order_number')):
        rows_to_add[i].append(c.get_text())
    
    return rows_to_add
    
def round_parser(show_round):
    jeopardy_round = show_body.find('div', id=show_round)
    categories = get_categories(jeopardy_round)   
    rows_to_add = clue_parser(jeopardy_round)
    
    colnames = ['Category', 'Clue', 'Correct Response', 'Answered Correctly', 'Triple Stumper', 'All Wrong Answers', 'Clue Value', 'Daily Double', 'Selection Order']
    show_df = pd.DataFrame(columns=colnames)
    
    for row in rows_to_add:
        show_df = show_df.append(pd.Series(row, index=colnames), ignore_index=True)
        
    show_df['Episode'] = episode
    show_df['Date First Aired'] = date
    show_df['Round'] = jeopardy_round.find('h2').get_text()
    
    return show_df
    
def contestants_parser(show_contestants):
    contestants = {}
    name_lookup = {}
    
    for c in show_contestants.find_all('p', class_='contestants'):
        name, bio = c.get_text().split(', ', 1)
        
        if 'originally' in bio:
            occupation, place = bio.split(' originally from ')
        else:
            occupation, place = bio.split(' from ')
        if '(' in place:
            place = place.split(' (', 1)[0]
            
        contestants[name] = (occupation, place)
        name_lookup[name.split()[0]] = name
        
    return name_lookup, contestants

def show_parser(season_df, url):
    show_page = requests.get(url)
    parsed_show = BeautifulSoup(show_page.content, 'html.parser')
    show_body = parsed_show.find('div', id='content')
    episode, date = show_body.find('h1').get_text().split(' - ')
    show_contestants = show_body.find('div', id='contestants')
    name_lookup, contestants = contestants_parser(show_contestants)
    for show_round in ['jeopardy_round', 'double_jeopardy_round']:
        season_df = pd.concat([season_df, round_parser(show_round)], ignore_index=True)
        
    return season_df  

def season_parser(show_links):
    colnames = ['Category', 'Clue', 'Correct Response', 'Answered Correctly', 'Triple Stumper', 'All Wrong Answers', 'Clue Value', 'Daily Double', 'Selection Order', 'Episode', 'Date First Aired', 'Round']
    season_df = pd.DataFrame(columns=colnames)
    
    for url in show_links[::-1]:
        season_df = show_parser(season_df, url)
        
    return season_df

In [107]:
# season1_df = season_parser(show_links)
season1_df.tail()

,Category,Clue,Correct Response,Answered Correctly,Triple Stumper,All Wrong Answers,Clue Value,Daily Double,Selection Order,Episode,Date First Aired,Round
2387,Trivia,First president who was a college graduate,John Adams,None,True,False,$1000,False,12,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
2388,Hunting,"D.L. Coburn's Pulitzer Prize-winning play, bas...",The Gin Game,Mark Singer,False,False,$1000,False,22,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
2389,History,"In '66, he took over government from Sukarno, ...",(General) Suharto,Mark Singer,False,False,$1000,False,7,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
2390,Automobiles,"Affecting both people & animals, it was the 1s...",anthrax,None,True,False,$1000,False,24,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
2391,Silly Songs,Diminutive actor who stood on a box to kiss So...,Alan Ladd,Joan Kantor,False,False,$1000,False,15,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round


In [104]:
colnames = ['Category', 'Clue', 'Correct Response', 'Answered Correctly', 'Triple Stumper', 'All Wrong Answers', 'Clue Value', 'Daily Double', 'Selection Order', 'Episode', 'Date First Aired', 'Round']
season_df = pd.DataFrame(columns=colnames)
season_df = show_parser(season_df, 'https://www.j-archive.com/showgame.php?game_id=7072')
season_df.tail()

,Category,Clue,Correct Response,Answered Correctly,Triple Stumper,All Wrong Answers,Clue Value,Daily Double,Selection Order,Episode,Date First Aired,Round
41,Trivia,First president who was a college graduate,John Adams,None,True,False,$1000,False,12,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
42,Hunting,"D.L. Coburn's Pulitzer Prize-winning play, bas...",The Gin Game,Mark Singer,False,False,$1000,False,22,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
43,History,"In '66, he took over government from Sukarno, ...",(General) Suharto,Mark Singer,False,False,$1000,False,7,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
44,Automobiles,"Affecting both people & animals, it was the 1s...",anthrax,None,True,False,$1000,False,24,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round
45,Silly Songs,Diminutive actor who stood on a box to kiss So...,Alan Ladd,Joan Kantor,False,False,$1000,False,15,Show #165,"Friday, April 26, 1985",Double Jeopardy! Round


In [81]:
contestants = {}
name_lookup = {}
show_page = requests.get('https://www.j-archive.com/showgame.php?game_id=7072')
parsed_show = BeautifulSoup(show_page.content, 'html.parser')
show_body = parsed_show.find('div', id='content')
episode, date = show_body.find('h1').get_text().split(' - ')
show_contestants = show_body.find('div', id='contestants')
for c in show_contestants.find_all('p', class_='contestants'):
    name, bio = c.get_text().split(', ', 1)
    if 'originally' in bio:
        occupation, place = bio.split(' originally from ')
    else:
        occupation, place = bio.split(' from ')
    if '(' in place:
        place = place.split(' (', 1)[0]
    contestants[name] = (occupation, place)
    name_lookup[name.split()[0]] = name
print(contestants)
print(name_lookup)

{'Mark Singer': ('an attorney and Sunday school teacher', 'Los Angeles, California'), 'Joan Kantor': ('a controller', 'Brooklyn, New York'), 'Doug Miller': ('an assignment editor', 'Houston, Texas')}
{'Mark': 'Mark Singer', 'Joan': 'Joan Kantor', 'Doug': 'Doug Miller'}


In [82]:
categories = []
jeopardy_round = show_body.find('div', id='jeopardy_round')
for c in jeopardy_round.find_all('td', class_='category_name'):
    categories.append(c.get_text().title())
categories

['U.S. Cities', 'Trivia', 'Hunting', 'History', 'Automobiles', 'Silly Songs']

In [83]:
rows_to_add = []
for c in jeopardy_round.find_all('td', class_='clue_text'):
    rows_to_add.append([categories[int(c.get('id')[-3:-2])-1], c.get_text()])
rows_to_add

[['U.S. Cities',
  'It recently moved ahead of Chicago to become our 2nd largest'],
 ['Trivia',
  'In 1796, this Frenchman confessed, "When I see an empty throne I feel the urge to sit on it"'],
 ['History',
  '13,000 houses were destroyed, but no one died in this city\'s "Great Fire" of 1666'],
 ['Silly Songs', 'This itsy bitsy creature "climbed up a waterspout"'],
 ['U.S. Cities',
  "There's no evidence that bison ever grazed in vicinity of this second largest NY city"],
 ['Trivia',
  'Made of metal, they consist of a point, a shank & a head, & are measured in pennies or inches'],
 ['History', 'East Jerusalem was part of this country from 1948-67'],
 ['Silly Songs', 'Jimmy does it, but "I don\'t care"'],
 ['U.S. Cities',
  'It has more Italians than Venice, more Irish than Dublin & more Jews than Jerusalem'],
 ['Trivia',
  'Country whose people all get their last names from the 438 words in the poem "Po-Chia-Hsing"'],
 ['History',
  'War with which this song was associated:"Over ther

In [84]:
for i, c in enumerate(jeopardy_round.find_all('div', onmouseover=True)):
    c = BeautifulSoup(c.get('onmouseover'))
    rows_to_add[i].append(c.find('em', class_='correct_response').get_text())
    if c.find('td', class_='right'):
        rows_to_add[i].append(name_lookup[c.find('td', class_='right').get_text()])
    else:
        rows_to_add[i].append(None)
    if c.find_all('td', class_='wrong') and c.find_all('td', class_='wrong')[-1].get_text() == 'Triple Stumper':
        rows_to_add[i].append(True)
    else:
        rows_to_add[i].append(False)
    if c.find_all('td', class_='wrong') and len(c.find_all('td', class_='wrong')) == 4:
        rows_to_add[i].append(True)
    else:
        rows_to_add[i].append(False)
rows_to_add

[['U.S. Cities',
  'It recently moved ahead of Chicago to become our 2nd largest',
  'Los Angeles',
  'Doug Miller',
  False,
  False],
 ['Trivia',
  'In 1796, this Frenchman confessed, "When I see an empty throne I feel the urge to sit on it"',
  'Napoleon',
  'Doug Miller',
  False,
  False],
 ['History',
  '13,000 houses were destroyed, but no one died in this city\'s "Great Fire" of 1666',
  'London',
  None,
  True,
  False],
 ['Silly Songs',
  'This itsy bitsy creature "climbed up a waterspout"',
  'the spider',
  'Mark Singer',
  False,
  False],
 ['U.S. Cities',
  "There's no evidence that bison ever grazed in vicinity of this second largest NY city",
  'Buffalo',
  'Doug Miller',
  False,
  False],
 ['Trivia',
  'Made of metal, they consist of a point, a shank & a head, & are measured in pennies or inches',
  'nails',
  'Mark Singer',
  False,
  False],
 ['History',
  'East Jerusalem was part of this country from 1948-67',
  'Jordan',
  'Mark Singer',
  False,
  False],
 ['Sil

In [85]:
for i, c in enumerate(jeopardy_round.find_all('table', class_='clue_header')):
    if c.find('td', class_='clue_value'):
        rows_to_add[i].append(c.find('td', class_='clue_value').get_text())
        rows_to_add[i].append(False)
    else:
        rows_to_add[i].append(c.find('td', class_='clue_value_daily_double').get_text())
        rows_to_add[i].append(True)
for i, c in enumerate(jeopardy_round.find_all('td', class_='clue_order_number')):
    rows_to_add[i].append(c.get_text())
    
print(rows_to_add)

[['U.S. Cities', 'It recently moved ahead of Chicago to become our 2nd largest', 'Los Angeles', 'Doug Miller', False, False, '$100', False, '3'], ['Trivia', 'In 1796, this Frenchman confessed, "When I see an empty throne I feel the urge to sit on it"', 'Napoleon', 'Doug Miller', False, False, '$100', False, '11'], ['History', '13,000 houses were destroyed, but no one died in this city\'s "Great Fire" of 1666', 'London', None, True, False, '$100', False, '1'], ['Silly Songs', 'This itsy bitsy creature "climbed up a waterspout"', 'the spider', 'Mark Singer', False, False, '$100', False, '16'], ['U.S. Cities', "There's no evidence that bison ever grazed in vicinity of this second largest NY city", 'Buffalo', 'Doug Miller', False, False, '$200', False, '4'], ['Trivia', 'Made of metal, they consist of a point, a shank & a head, & are measured in pennies or inches', 'nails', 'Mark Singer', False, False, '$200', False, '12'], ['History', 'East Jerusalem was part of this country from 1948-67',

In [86]:
colnames = ['Category', 'Clue', 'Correct Response', 'Answered Correctly', 'Triple Stumper', 'All Wrong Answers', 'Clue Value', 'Daily Double', 'Selection Order']
show_df = pd.DataFrame(columns=colnames)
for row in rows_to_add:
    show_df = show_df.append(pd.Series(row, index=colnames), ignore_index=True)
show_df.tail()

,Category,Clue,Correct Response,Answered Correctly,Triple Stumper,All Wrong Answers,Clue Value,Daily Double,Selection Order
15,Silly Songs,"Size shoe worn by ""My Darling Clementine""",No. 9,Mark Singer,False,False,$400,False,19
16,U.S. Cities,"The mapmakers' lines which divide Washington, ...",the Capitol,Mark Singer,False,False,$500,False,7
17,Trivia,What the letters in the name of the cable chan...,the Entertainment Sports Programming Network,Doug Miller,False,False,$500,False,15
18,History,"In 1844, this country gained its independence ...",the Dominican Republic,Doug Miller,False,False,$500,False,10
19,Silly Songs,Where K-K-K-Katy will be waiting,at the k-k-k-kitchen door,None,True,False,$500,False,20


In [87]:
show_df['Episode'] = episode
show_df['Date First Aired'] = date
show_df['Round'] = jeopardy_round.find('h2').get_text()
show_df.head()

,Category,Clue,Correct Response,Answered Correctly,Triple Stumper,All Wrong Answers,Clue Value,Daily Double,Selection Order,Episode,Date First Aired,Round
0,U.S. Cities,It recently moved ahead of Chicago to become o...,Los Angeles,Doug Miller,False,False,$100,False,3,Show #165,"Friday, April 26, 1985",Jeopardy! Round
1,Trivia,"In 1796, this Frenchman confessed, ""When I see...",Napoleon,Doug Miller,False,False,$100,False,11,Show #165,"Friday, April 26, 1985",Jeopardy! Round
2,History,"13,000 houses were destroyed, but no one died ...",London,None,True,False,$100,False,1,Show #165,"Friday, April 26, 1985",Jeopardy! Round
3,Silly Songs,"This itsy bitsy creature ""climbed up a watersp...",the spider,Mark Singer,False,False,$100,False,16,Show #165,"Friday, April 26, 1985",Jeopardy! Round
4,U.S. Cities,There's no evidence that bison ever grazed in ...,Buffalo,Doug Miller,False,False,$200,False,4,Show #165,"Friday, April 26, 1985",Jeopardy! Round
